In [1]:
"""
mojimojiくんで前処理をしよう
"""
import pandas as pd
import mojimoji

# 食材名データの読み込み
data = pd.read_csv("./data/fixed_name_data.csv",dtype="str")

# 空欄のデータは削除
print("生データの総数",len(data))
data = data.dropna(subset=["result"])
print("削除後",len(data))

# idをもとに並び替え
data = data.sort_values("id")

# 欠損地データの置き換え
data = data.fillna("わからん")

# 半角数字を全角に
data["quantity"] = data["quantity"].apply(mojimoji.han_to_zen)

生データの総数 10895692
削除後 10895692


In [2]:
"""
大さじなどの接頭語の数詞ように調整
"""
import MeCab

mecab = MeCab.Tagger("-d /var/lib/mecab/dic/ipadic_latest/ -u ./data/user_dic.dic")

text = '１/１０'
mecab.parse('') # 文字列がGCされるのを防ぐ
node = mecab.parseToNode(text)

while node:
    # 単語を取得
    word = node.surface
    # 品詞を取得
    pos = node.feature.split(",")
    print('{0} , {1}'.format(word, pos))
    # 次の単語に進める
    node = node.next

 , ['BOS/EOS', '*', '*', '*', '*', '*', '*', '*', '*']
１ , ['名詞', '数', '*', '*', '*', '*', '１', 'イチ', 'イチ']
/ , ['名詞', 'サ変接続', '*', '*', '*', '*', '*']
１ , ['名詞', '数', '*', '*', '*', '*', '１', 'イチ', 'イチ']
０ , ['名詞', '数', '*', '*', '*', '*', '０', 'ゼロ', 'ゼロ']
 , ['BOS/EOS', '*', '*', '*', '*', '*', '*', '*', '*']


In [3]:
"""
通常のmecabでの実装
処理手順：
1.mecab（NEologdではなく標準の辞書）で品詞ごとに分割
2.品詞によって以下の処理を行う
"""
def pick_amount(raw_amount,m):
  """
  seq_candidate_amount
  変換候補となる数詞を格納
  複数の数詞で構成される語（例：12）などは
  数詞以外の語が来るまでを連結して格納
  区切りは＊
  """
  seq_candidate_amount = []

  """
  実際に数字に変換する文字を格納
  """
  #done_amount = []

  parsed_text = m.parseToNode(raw_amount) # mecabで形態素に分解
  tmp_diget = ""                          # 処理中の数字を格納
  operator = ""                           # /などの演算子を含む場合の判定
  
  try:
    while parsed_text:
      node = parsed_text.feature.split(',')
      if node[1] == "数":
        # 演算子の処理
        if len(operator) > 0 and len(tmp_diget) > 0:
          tmp_diget += operator
          operator = ""

        tmp_diget += parsed_text.surface
      elif node[2] == "助数詞":
        if len(tmp_diget) !=0:
          tmp_diget += ';' + node[6]
          seq_candidate_amount.append(tmp_diget)
          tmp_diget = ""
        else:
          pass
      else:
        if node[6] == "/" or node[6] == "／":
          operator = node[6]
        else:
          tmp_diget = ""
      
      parsed_text = parsed_text.next
    return "_".join(seq_candidate_amount)
  except TypeError:
    return "undefined"

In [4]:
print(pick_amount("１／２〜１／４個",m=mecab))

１／４;個


In [5]:
from tqdm import tqdm as progress
# 進捗を確認
progress.pandas()
# 関数を適応
data['amount'] = data['quantity'].progress_apply(pick_amount,m=mecab)
print(len(data[data["amount"].str.contains(';',na=False)])/len(data))
data.head(50)

100%|██████████| 10895692/10895692 [02:29<00:00, 72876.14it/s]


0.5092909197506684


,Unnamed: 0,id,result,quantity,amount
2248976,2248976,000005f43fc4c95a43eb061be8d06fadc205bbfe,パスタ,１人前,１;人前
2248977,2248977,000005f43fc4c95a43eb061be8d06fadc205bbfe,マヨネーズ,大さじ１,
2248978,2248978,000005f43fc4c95a43eb061be8d06fadc205bbfe,塩,少々,
2248975,2248975,000005f43fc4c95a43eb061be8d06fadc205bbfe,きゅうり,１本,１;本
2248974,2248974,000005f43fc4c95a43eb061be8d06fadc205bbfe,じゃがいも,２〜３個,３;個
6410021,6410021,00001944477645056f44a8cc3472d033b4cb3ffa,薄力粉,大３,
6410019,6410019,00001944477645056f44a8cc3472d033b4cb3ffa,砂糖,１５０ｇ,１５０;g
6410018,6410018,00001944477645056f44a8cc3472d033b4cb3ffa,サワークリーム,１００ｇ,１００;g
6410017,6410017,00001944477645056f44a8cc3472d033b4cb3ffa,クリームチーズ,３５０ｇ,３５０;g
6410022,6410022,00001944477645056f44a8cc3472d033b4cb3ffa,コーンスターチ,大１,


In [6]:
"""
通常のmecabでの実装
処理手順：
1.mecab（NEologdではなく標準の辞書）で品詞ごとに分割
2.品詞によって以下の処理を行う
"""
def pick_amount_sazi(raw_amount,m):
  """
  seq_candidate_amount
  変換候補となる数詞を格納
  複数の数詞で構成される語（例：12）などは
  数詞以外の語が来るまでを連結して格納
  区切りは＊
  """
  seq_candidate_amount = []

  """
  実際に数字に変換する文字を格納
  """
  #done_amount = []

  parsed_text = m.parseToNode(raw_amount) # mecabで形態素に分解
  tmp_diget = ""                          # 処理中の数字を格納
  sazi = ""
  operator = ""                           # /などの演算子を含む場合の判定
  
  try:
    while parsed_text:
      node = parsed_text.feature.split(',')
      if node[1] == "数" or node[6] == "／" or node[1] == "サ変接続":
        tmp_diget += parsed_text.surface

      elif node[2] == "助数詞":
        tmp_diget = ""
        sazi = node[6]
      else:
        if len(sazi) != 0 and len(tmp_diget) != 0:
          tmp_diget +=";" + sazi
          seq_candidate_amount.append(tmp_diget)
          sazi = ""
          
      parsed_text = parsed_text.next
    return "_".join(seq_candidate_amount)
  except TypeError:
    return "undefined"

In [7]:
print(pick_amount_sazi("小1/２",m=mecab))

1/２;小さじ


In [8]:
"""
おおさじなどの単位に対応
"""

sazi_data = data[~data["amount"].str.contains(';')]
sazi_data = sazi_data.drop("amount",axis=1)
sazi_data['quantity'] = sazi_data['quantity'].progress_apply(mojimoji.zen_to_han,kana=False,ascii=False)
sazi_data["amount"] = sazi_data['quantity'].progress_apply(pick_amount_sazi,m=mecab)
print( (len(sazi_data[sazi_data["amount"].str.contains(';')])) / len(data) )
sazi_data.head(50)

100%|██████████| 5346615/5346615 [01:03<00:00, 84422.57it/s]


0.24464201080573864


,Unnamed: 0,id,result,quantity,amount
2248977,2248977,000005f43fc4c95a43eb061be8d06fadc205bbfe,マヨネーズ,大さじ1,1;大さじ
2248978,2248978,000005f43fc4c95a43eb061be8d06fadc205bbfe,塩,少々,
6410021,6410021,00001944477645056f44a8cc3472d033b4cb3ffa,薄力粉,大3,3;大さじ
6410022,6410022,00001944477645056f44a8cc3472d033b4cb3ffa,コーンスターチ,大1,1;大さじ
10814724,3231391,00001944477645056f44a8cc3472d033b4cb3ffa,empty,大1．5,1．5;大さじ
4933049,4933049,000021be4ed94ff1875fe6415e98931104ff0e85,醤油,小さじ1,1;小さじ
10068434,2485101,000021be4ed94ff1875fe6415e98931104ff0e85,油揚げ,1／10,
4933050,4933050,000021be4ed94ff1875fe6415e98931104ff0e85,みりん,小さじ1,1;小さじ
10068435,2485102,000021be4ed94ff1875fe6415e98931104ff0e85,こねぎ,少々,
4933051,4933051,000021be4ed94ff1875fe6415e98931104ff0e85,砂糖,小さじ1,1;小さじ


In [9]:
print( ( len(sazi_data[sazi_data["amount"].str.contains(';')]) + len(data[data["amount"].str.contains(';',na=False)]) ) / len(data))

0.7539329305564071


In [10]:
"""
現時点での最終出力データを作成
"""
output = pd.concat([sazi_data,data[data["amount"].str.contains(';')]])
print(len(output))
output = output.sort_values("id")
output.to_csv("./data/fix_data.csv")
output.head(50)

10895692


,Unnamed: 0,id,result,quantity,amount
2248977,2248977,000005f43fc4c95a43eb061be8d06fadc205bbfe,マヨネーズ,大さじ1,1;大さじ
2248974,2248974,000005f43fc4c95a43eb061be8d06fadc205bbfe,じゃがいも,２〜３個,３;個
2248975,2248975,000005f43fc4c95a43eb061be8d06fadc205bbfe,きゅうり,１本,１;本
2248976,2248976,000005f43fc4c95a43eb061be8d06fadc205bbfe,パスタ,１人前,１;人前
2248978,2248978,000005f43fc4c95a43eb061be8d06fadc205bbfe,塩,少々,
6410018,6410018,00001944477645056f44a8cc3472d033b4cb3ffa,サワークリーム,１００ｇ,１００;g
6410017,6410017,00001944477645056f44a8cc3472d033b4cb3ffa,クリームチーズ,３５０ｇ,３５０;g
6410020,6410020,00001944477645056f44a8cc3472d033b4cb3ffa,卵,３個,３;個
6410019,6410019,00001944477645056f44a8cc3472d033b4cb3ffa,砂糖,１５０ｇ,１５０;g
6410021,6410021,00001944477645056f44a8cc3472d033b4cb3ffa,薄力粉,大3,3;大さじ


In [37]:
"""
品詞のみ抽出して結合
"""

def pick_hinshi(words,m):
  output = []
  parsed_text = m.parseToNode(words)
  
  while parsed_text:
    node = parsed_text.feature.split(',')
    output.append(";".join(node[0:3]))
    parsed_text = parsed_text.next
  
  return "_".join(output)

In [ ]:
# 出現語彙の計測
tmp = sazi_data[~sazi_data["amount"].str.contains(';')]
data_rest = tmp["quantity"].value_counts()
data_rest.head(50)

In [38]:
data_hinshi = tmp["quantity"].progress_apply(pick_hinshi,m=mecab)
data_hinshi.value_counts().head(50)

100%|██████████| 3217144/3217144 [00:37<00:00, 85116.13it/s]


BOS/EOS;*;*_名詞;一般;*_BOS/EOS;*;*                                          975611
BOS/EOS;*;*_副詞;助詞類接続;*_BOS/EOS;*;*                                       566537
BOS/EOS;*;*_動詞;自立;*_助動詞;*;*_BOS/EOS;*;*                                  540111
BOS/EOS;*;*_名詞;副詞可能;*_BOS/EOS;*;*                                        233020
BOS/EOS;*;*_接頭詞;名詞接続;*_名詞;一般;*_助詞;格助詞;一般_BOS/EOS;*;*                      97919
BOS/EOS;*;*_名詞;数;*_名詞;一般;*_BOS/EOS;*;*                                    72597
BOS/EOS;*;*_名詞;数;*_名詞;サ変接続;*_BOS/EOS;*;*                                  62725
BOS/EOS;*;*_名詞;形容動詞語幹;*_助動詞;*;*_助詞;副助詞;*_BOS/EOS;*;*                      52146
BOS/EOS;*;*_接頭詞;名詞接続;*_名詞;一般;*_BOS/EOS;*;*                                45476
BOS/EOS;*;*_名詞;数;*_記号;一般;*_名詞;数;*_BOS/EOS;*;*                             30259
BOS/EOS;*;*_名詞;数;*_助動詞;*;*_BOS/EOS;*;*                                    24142
BOS/EOS;*;*_名詞;形容動詞語幹;*_BOS/EOS;*;*                                       17151
BOS/EOS;*;*_接頭詞;名詞接続;*_名詞;一般;*_助詞;連体化;*_

In [21]:
print(" ".join(tmp["quantity"].values.tolist()[0:10]))


お好みで お好みで たくさん（安い日本酒でいい） 適量（味見しながらね） 4つ わからん 少々 少々 適当 イングリッシュブレクファスト


In [25]:
# wordcloudで可視化
from wordcloud import WordCloud

text = WordCloud.__doc__
wc = WordCloud(width=480, height=320,background_color="white",font_path=".//SourceHanSerifK-Light.otf")
wc.generate(" ".join(tmp["quantity"].values.tolist()))
wc.to_file('ingredient_amount.png')